## Import libraries

In [1]:
# standard setup
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import time

# useful helper function
from helpers import imshow

## Adding Dense Optical Flow on Video Frame

In [176]:
# parameters for farneback optical flow
fb_params = dict( pyr_scale = 0.5, 
                  levels = 3, 
                  winsize = 5, 
                  iterations = 3, 
                  poly_n = 5,
                  poly_sigma = 1.2, 
                  flags = 0 )

In [177]:
def initializeCamera(w):
    '''
    @w: width of the video frame
    @returns: camera object
    '''
    camera = cv2.VideoCapture(0)
    camera.set(cv2.CAP_PROP_FRAME_WIDTH, w) 
    camera.set(cv2.CAP_PROP_FRAME_HEIGHT, w * 3/4) 
    camera.set(cv2.CAP_PROP_EXPOSURE,-4) 
    return camera

In [73]:
def draw_motion_field(u, v, scale, stride, color=(0, 255, 0)):
    """Draws motion fields given the displacements in X and Y directions.
    
    """
    img_out = np.zeros((v.shape[0], u.shape[1], 3), dtype=np.uint8)

    for y in range(0,v.shape[0],stride):
        for x in range(0,u.shape[1],stride):
            cv2.line(img_out, (x, y), (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), color, 1)
            cv2.circle(img_out, (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), 1, color, 1)
            
    return img_out

In [205]:
def draw_motion_field_on_frame(frame, u, v, scale, stride, color=(0, 0, 0)):
    """Draws motion fields given the displacements in X and Y directions.
    
    """
    img_out = frame.copy()

    for y in range(0,v.shape[0],stride):
        for x in range(0,u.shape[1],stride):
            cv2.line(img_out, (x, y), (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), color, 2)
            cv2.circle(img_out, (x + int(u[y, x] * scale), y + int(v[y, x] * scale)), 3, color, 3)
            
    return img_out

In [204]:
from IPython.display import clear_output

# Initialize camera
camera = initializeCamera(640)

# Flip frame
ret, prev_frame = camera.read()
prev_frame = cv2.flip(prev_frame, 1)
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

while True:
    clear_output(True)

    # Get current frame at flip it
    ret, next_frame = camera.read()
    next_frame = cv2.flip(next_frame, 1)
    next_gray = cv2.cvtColor(next_frame, cv2.COLOR_BGR2GRAY)

    # Calculate the dense optical flow
    flow = cv2.calcOpticalFlowFarneback(prev_gray, next_gray, None, **fb_params)
    
    # Obtain the flow magnitude and direction angle
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    
    # Add vector field to a video frame
    field_img = draw_motion_field_on_frame(next_frame, flow[...,0], flow[...,1], scale=2, stride=50)
    
    print flow[...,0].shape == flow[...,1].shape
    
    cv2.imshow('Frame', next_frame)
    cv2.imshow('Field', field_img)
    
    prev_gray = next_gray
    
    if cv2.waitKey(5) == 27:
        break      
        
cv2.destroyAllWindows()
camera.release()
cv2.waitKey(1) # extra waitKey sometimes needed to close camera window

True


-1